In [1]:
from flask import Flask, request, jsonify
from threading import Thread
import json

In [2]:
import os
from openai import OpenAI
import json
from vllm import SamplingParams

INFO 07-13 06:48:51 [__init__.py:248] Automatically detected platform rocm.


In [3]:
BASE_URL = f"http://localhost:3000/v1"
MODEL_PATH = "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat"

In [4]:
client = OpenAI(api_key="NULL", base_url=BASE_URL, timeout=None, max_retries=0)

In [5]:
import json
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil import tz
class AI_AGENT:
    def __init__(self, client, MODEL_PATH):
        self.base_url = "[invalid url]"  # Placeholder
        self.model_path = MODEL_PATH
        self.client = client

    def give_time(self, json_input):
        # Parse JSON input
        data = json.loads(json_input) if isinstance(json_input, str) else json_input
        subject = data.get("Subject", "")
        email_content = data.get("EmailContent", "")
        datetime_str = data.get("Datetime", "")
        from_email = data.get("From", "")
        attendees = data.get("Attendees", [])

        # Add From to attendees if not present
        if from_email and not any(a["email"] == from_email for a in attendees):
            attendees.append({"email": from_email, "events": []})

        # Convert datetime_str to datetime object with IST timezone
        try:
            start_dt = datetime.strptime(datetime_str, "%d-%m-%YT%H:%M:%S").replace(tzinfo=tz.gettz("Asia/Kolkata"))
            end_dt = start_dt + timedelta(days=14)
        except ValueError:
            return {"error": "Invalid datetime format. Expected DD-MM-YYYYTHH:MM:SS"}

        # Use LLM to get severity and duration
        prompt = f"""
        Analyze the following email content to determine the severity of the issue (low, medium, high) and suggest a suitable duration in minutes for a meeting to discuss it. The duration should be between 15 and 120 minutes. Return your answer in JSON format: {{"severity": "low|medium|high", "duration": <minutes>}}.

        Subject: {subject}
        Content: {email_content}
        """
        try:
            response = self.client.chat.completions.create(
                model=self.model_path,
                temperature=0.0,
                messages=[{"role": "user", "content": prompt}]
            )
            llm_output = response.choices[0].message.content
            llm_result = json.loads(llm_output)
            severity = llm_result["severity"]
            duration_minutes = int(llm_result["duration"])
            if severity not in ["low", "medium", "high"] or not (15 <= duration_minutes <= 120):
                raise ValueError("Invalid LLM response")
        except Exception as e:
            # Fallback
            severity = "low"  # Based on "status update" and "30 minutes"
            duration_minutes = 30
            print(f"LLM error: {str(e)}")

        # Find a free slot
        def is_working_hours(dt):
            ist = tz.gettz("Asia/Kolkata")
            dt_ist = dt.astimezone(ist)
            if dt_ist.weekday() >= 5:  # Saturday or Sunday
                return False
            hour = dt_ist.hour
            minute = dt_ist.minute
            return 9 <= hour < 18 or (hour == 18 and minute == 0)

        def has_conflict(start, end, events):
            for event in events:
                if event.get("Summary") in ["Off Hours", "Weekend"]:
                    continue
                event_start = parse(event["StartTime"])
                event_end = parse(event["EndTime"])
                if not (end <= event_start or start >= event_end):
                    return True
            return False

        current_dt = start_dt
        duration = timedelta(minutes=duration_minutes)
        found_slot = False
        max_search_dt = end_dt + timedelta(days=3) 

        while current_dt < max_search_dt:
            slot_end = current_dt + duration
            if is_working_hours(current_dt):
                conflict = False
                for attendee in attendees:
                    events = attendee.get("events", [])
                    if has_conflict(current_dt, slot_end, events):
                        conflict = True
                        break
                if not conflict:
                    found_slot = True
                    start_time = current_dt.isoformat()
                    end_time = slot_end.isoformat()
                    reason = f"Scheduled meeting at {start_time} for {duration_minutes} minutes based on {severity} severity. This slot is within working hours (9:00 AM to 6:00 PM IST, Monday to Friday) and has no conflicts for all participants."
                    if current_dt > end_dt:
                        reason += " Note: This slot is outside the initial 14-day range but is the earliest available."
                    break
            current_dt += timedelta(minutes=30)

        if not found_slot:
            return {
                "severity": severity,
                "duration": str(duration_minutes),
                "start_time": "",
                "end_time": "",
                "reason": "No available slot found within 30 days after the end date."
            }

        return {
            "severity": severity,
            "duration": str(duration_minutes),
            "start_time": start_time,
            "end_time": end_time,
            "reason": reason
        }

In [6]:
ai_agent = AI_AGENT(client, MODEL_PATH)

In [7]:
import json
from datetime import datetime, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "./Keys/" + user.split("@")[0] + ".token"
    try:
        user_creds = Credentials.from_authorized_user_file(token_path)
        calendar_service = build("calendar", "v3", credentials=user_creds)
        events_result = calendar_service.events().list(
            calendarId='primary',
            timeMin=start,
            timeMax=end,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])
        
        for event in events:
            attendee_list = []
            try:
                for attendee in event.get("attendees", []):
                    attendee_list.append(attendee['email'])
            except:
                attendee_list.append("SELF")
            # Handle both dateTime and date fields
            start_time = event["start"].get("dateTime", event["start"].get("date", ""))
            end_time = event["end"].get("dateTime", event["end"].get("date", ""))
            if not start_time or not end_time:
                continue  # Skip events with invalid time formats
            events_list.append({
                "StartTime": start_time,
                "EndTime": end_time,
                "NumAttendees": len(set(attendee_list)),
                "Attendees": list(set(attendee_list)),
                "Summary": event.get("summary", "")
            })
    except FileNotFoundError:
        events_list.append({"error": f"Token file not found for {user}"})
    except Exception as e:
        events_list.append({"error": f"Failed to retrieve events for {user}: {str(e)}"})
    return events_list

def preprocess(json_input):
    # Parse JSON input
    data = json.loads(json_input) if isinstance(json_input, str) else json_input.copy()
    
    # Extract Datetime and compute time range
    start_time_str = data.get("Datetime", "")
    try:
        start_dt = datetime.strptime(start_time_str, "%d-%m-%YT%H:%M:%S")
        start_time_rfc3339 = start_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
        end_dt = start_dt + timedelta(days=14)
        end_time_rfc3339 = end_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    except ValueError:
        return {"error": "Invalid datetime format. Expected DD-MM-YYYYTHH:MM:SS"}
    
    # Extract Attendees
    attendees = data.get("Attendees", [])
    
    # Update events for each attendee
    for attendee in attendees:
        email = attendee.get("email", "")
        if not email:
            attendee["events"] = []
            continue
        # Fetch events for the user
        events = retrive_calendar_events(email, start_time_rfc3339, end_time_rfc3339)
        # Filter out error entries
        valid_events = [event for event in events if "error" not in event]
        attendee["events"] = valid_events
    
    return data

In [8]:
import json

def append_json(json1, json2):

    data1 = json.loads(json1) if isinstance(json1, str) else json1.copy()
    result = data1.copy()

    result["start_time"] = json2["start_time"]
    result["end_time"] = json2["end_time"]
    result["duration"] = json2["duration"]

    result["metadata"] = {
        "severity": json2["severity"]
    }
    
    return result

In [9]:
app = Flask(__name__)
received_data = []

In [10]:
def your_meeting_assistant(data): 
    pre=preprocess(data)
    AI=ai_agent.give_time(pre)
    final=append_json(pre,AI)
    
    return final

In [11]:
@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")
    new_data = your_meeting_assistant(data)  # Your AI Meeting Assistant Function Call
    received_data.append(data)
    print(f"\n\n\n Sending:\n {json.dumps(new_data, indent=2)}")
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=5000)

In [12]:

Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://134.199.206.143:5000
Press CTRL+C to quit



 Received: {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "02-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "EmailContent": "Hi Team. Let's meet next Thursday and discuss about our Goals."
}


64.227.21.251 - - [13/Jul/2025 06:49:07] "POST /receive HTTP/1.1" 200 -


LLM error: Expecting value: line 1 column 1 (char 0)



 Sending:
 {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "02-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-02T18:00:00+05:30",
          "EndTime": "2025-07-03T09:00:00+05:30",
          "NumAttendees": 0,
          "Attendees": [],
          "Summary": "Off Hours"
        },
        {
          "StartTime": "2025-07-03T18:00:00+05:30",
          "EndTime": "2025-07-04T09:00:00+05:30",
          "NumAttendees": 0,
          "Attendees": [],
          "Summary": "Off Hours"
        },
        {
          "StartTime": "2025-07-04T18:00:00+05:30",
          "EndTime": "2025-07-05T09:00:00+05:30",
          "NumAttendees": 0,
          "Attendees": [],
          "Summary": "Off Hours"
        },
        {
          "StartTime": "2025-07-05",
          "

64.227.21.251 - - [13/Jul/2025 06:53:12] "POST /receive HTTP/1.1" 200 -





 Sending:
 {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033b5",
  "Datetime": "02-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-02T18:00:00+05:30",
          "EndTime": "2025-07-03T09:00:00+05:30",
          "NumAttendees": 0,
          "Attendees": [],
          "Summary": "Off Hours"
        },
        {
          "StartTime": "2025-07-03T18:00:00+05:30",
          "EndTime": "2025-07-04T09:00:00+05:30",
          "NumAttendees": 0,
          "Attendees": [],
          "Summary": "Off Hours"
        },
        {
          "StartTime": "2025-07-04T18:00:00+05:30",
          "EndTime": "2025-07-05T09:00:00+05:30",
          "NumAttendees": 0,
          "Attendees": [],
          "Summary": "Off Hours"
        },
        {
          "StartTime": "2025-07-05",
          "EndTime": "2025-07-06",
          "NumAttendees": 0,
